In [ ]:
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
from scipy.fft import fftfreq
from scipy.optimize import minimize
from util import (
    calc_phase_shift_for_antennae,
    dedisperse_dataset,
    get_demeaned_scrunched_data,
    get_fold_segment,
    get_folded_data,
    get_fourier_power_spectrum,
)

THETA_RADIANS = 0
PHI_RADIANS = np.pi / 2

REFERENCE_WAVELENGTH = 1_480_000_000  # MHz


mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

# Create a new MLflow Experiment
mlflow.set_experiment("2025-Beamforming-Python-Jimble")


# data = np.load('../all_data.npy)
antenna_locations = pd.read_csv("portmap.csv")
antenna_locations["comment"] = (
    antenna_locations["comment"].astype(str).str.replace("nan", "")
)

antenna_locations["x_loc"] /= 100
antenna_locations["y_loc"] /= 100

## antenna WEMs 27 / 28 are not used
antenna_locations = antenna_locations[~antenna_locations["wem_no"].isin([27, 28])]
antenna_locations["r"] = np.sqrt(
    antenna_locations["x_loc"] ** 2 + antenna_locations["y_loc"] ** 2
)


# Calculate VELA Period
vela_f0_s = 0.089328385024
vela_f1_s = 1.25008 * 10**-13
vela_fepoch_mjd = 51559.319
current_mjd = 60769

num_seconds_since_epoch = (60769 - 51559.319) * 24 * 60 * 60
vela_period_calc = vela_f0_s + vela_f1_s * num_seconds_since_epoch
VELA_PERIOD = vela_period_calc * 1000  # ms
vela_period_calc

In [ ]:
# I think the unit on x_loc and y_loc is cm and the reference point is the center of the array.
# How to deal with dual polarization in practice? Two images? One image with both?


# The order of the channels in the np array is the index of the row - not the
# astro_chan number.
antenna_locations

The wave vector is given by:
$$ 
    \hat{w} = (\cos(\theta)\cos(\phi), \sin(\theta)\cos(\phi), \sin(\phi))
$$

where I'm assuming this is the Az / Alt relative to the way that the dish is currently pointing.

If this is pointing directly at VELA - then this should give the maximum of the peak signal when we beamform.

In [ ]:
data = np.load("../all_data.npy")

## Drop all the channels which are unused
a = (
    antenna_locations[antenna_locations["comment"] != ""]
    .reset_index()["index"]
    .tolist()
)

data = np.delete(data, list([int(i) for i in a]), axis=0)

antenna_locations = antenna_locations[antenna_locations["comment"] == ""].reset_index(
    drop=True
)

# data and antenna_locations should now be aligned.

In [ ]:
# Calc antenna phase shift
antenna_phase_shift = calc_phase_shift_for_antennae(
    THETA_RADIANS, PHI_RADIANS, antenna_locations, REFERENCE_WAVELENGTH
)
antenna_phase_shift = antenna_phase_shift[np.newaxis, :]


# Calculate weights
rs = antenna_locations["r"].to_numpy()

# trial this Gaussian weighting scheme - sigma chosen arbitrarily.
SIGMA = 0.2
weights = np.exp(-(rs**2) / (2 * SIGMA**2))
weights = weights / weights.sum()  # make add to 1
weights = weights[np.newaxis, :]

In [ ]:
SEGMENT = 2**12
SAMPLING_RATE = int(51_200_000 / 27)  # Hz

OVERSAMPLED_BANDWIDTH = SAMPLING_RATE
BANDWIDTH = SAMPLING_RATE * 27 / 32
SKY_FREQUENCY = 926 * BANDWIDTH

In [ ]:
for SIGMA in range(1, 10):
    SIGMA /= 10
    weights = np.exp(-(rs**2) / (2 * SIGMA**2))
    weights = weights / weights.sum()  # make add to 1
    weights = weights[np.newaxis, :]
    with mlflow.start_run() as run:
        RUN_ID = run.info.run_id
        mlflow.log_params(
            {
                "window_size": SEGMENT,
                "sampling_rate": SAMPLING_RATE,
                "bandwidth": BANDWIDTH,
                "oversampled_bandwidth": OVERSAMPLED_BANDWIDTH,
                "theta_radians": THETA_RADIANS,
                "phi_radians": PHI_RADIANS,
                "reference_wavelength": REFERENCE_WAVELENGTH,
                "vela_period_ms": VELA_PERIOD,
                "weighting_scheme": "exponential",
                "weighting_sigma": SIGMA,
            }
        )

        # beam_formed_data = antenna_phase_shift @ data

        weighted_and_phase_shifted = weights * antenna_phase_shift
        weighted_beamformed_data = weighted_and_phase_shifted @ data

        # output = get_fourier_power_spectrum(beam_formed_data, SEGMENT)
        output_weighted = get_fourier_power_spectrum(weighted_beamformed_data, SEGMENT)

        FOLD_SEGMENT = get_fold_segment(VELA_PERIOD, 27 / 51_200_000, SEGMENT)
        # output_folded = get_folded_data(output, FOLD_SEGMENT,VELA_PERIOD, 27 / 51_200_000, SEGMENT)
        output_weighted_folded = get_folded_data(
            output_weighted, FOLD_SEGMENT, VELA_PERIOD, 27 / 51_200_000, SEGMENT
        )

        n_segments = (weighted_beamformed_data.shape[1] - SEGMENT) // (SEGMENT)

        frequencies_axis = fftfreq(SEGMENT, 1 / SAMPLING_RATE)
        frequencies_axis = np.fft.fftshift(frequencies_axis) + SKY_FREQUENCY
        frequencies_axis_mhz = frequencies_axis / 1_000_000
        times = np.arange(n_segments) * (SEGMENT) / int(SAMPLING_RATE)

        demeaned_data = get_demeaned_scrunched_data(output_weighted_folded)
        signal_to_noise = (
            demeaned_data.max() / demeaned_data[demeaned_data.shape[0] // 2 :].std()
        )

        mlflow.log_metric("signal_to_noise", signal_to_noise)

        fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True, sharey=True)
        # ax = axes[1]
        im = ax.plot(
            times[:FOLD_SEGMENT],
            demeaned_data,
        )

        ax.set_ylabel("Weighted Power (arb)")
        ax.set_xlabel("Time [s]")

        plt.suptitle(
            "Power vs Time for Un-decayed Beamformed Data (Folded)", fontsize=16
        )
        plt.savefig(f"run_{RUN_ID}.png")
        mlflow.log_artifact(f"run_{RUN_ID}.png")
        plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 16), sharex=True, sharey=True)

# ax = axes[0]
# im = ax.plot(
#     get_demeaned_scrunched_data(output),
# )
# ax.set_ylabel("Channel Frequency [Hz]")
# ax.set_xlabel("Time [s]")


# ax = axes[1]

im = ax.plot(
    get_demeaned_scrunched_data(output_weighted),
)
ax.set_ylabel("Channel Frequency [Hz]")
ax.set_xlabel("Time [s]")

plt.suptitle("Frequency vs Time for Multiple Channels", fontsize=16)

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True, sharey=True)

# ax = axes[0]

# im = ax.plot(
#     times[:FOLD_SEGMENT],
#     get_demeaned_scrunched_data(output_folded),
# )
# ax.set_ylabel("Power (arb)")
# ax.set_xlabel("Time [s]")


demeaned_data = get_demeaned_scrunched_data(output_weighted_folded)

# ax = axes[1]
im = ax.plot(
    times[:FOLD_SEGMENT],
    demeaned_data,
)


ax.set_ylabel("Weighted Power (arb)")
ax.set_xlabel("Time [s]")


plt.suptitle("Power vs Time for Un-decayed Beamformed Data (Folded)", fontsize=16)

plt.show()
print(demeaned_data.max() / demeaned_data[demeaned_data.shape[0] // 3 :].std())

In [ ]:
## Look at getting peak signal-to-noise


def max_func(x):
    return -dataset_to_optimize(x).max() * 1000


def dataset_to_optimize(x):
    theta_radians = THETA_RADIANS + x[0]
    phi_radians = PHI_RADIANS + x[1]

    antenna_phase_shift = calc_phase_shift_for_antennae(
        theta_radians, phi_radians, antenna_locations, REFERENCE_WAVELENGTH
    )
    antenna_phase_shift = antenna_phase_shift[np.newaxis, :]
    # Calculate weights
    rs = antenna_locations["r"].to_numpy()

    # trial this Gaussian weighting scheme - sigma chosen arbitrarily.
    SIGMA = 1 + x[2]
    weights = np.exp(-(rs**2) / (2 * SIGMA**2))
    weights = weights[np.newaxis, :]

    weighted_and_phase_shifted = weights * antenna_phase_shift
    weighted_beamformed_data = weighted_and_phase_shifted @ data
    output_weighted = get_fourier_power_spectrum(weighted_beamformed_data, SEGMENT)
    output_weighted_folded = get_folded_data(
        output_weighted, VELA_PERIOD, 27 / 51_200_000, SEGMENT
    )

    output_weighted_folded_power = get_demeaned_scrunched_data(output_weighted_folded)
    std_noise = output_weighted_folded_power[
        output_weighted_folded_power.shape[0] // 2 :
    ].std(axis=0)

    output_weighted_folded_power /= std_noise

    return output_weighted_folded_power


# x = [0, 0, 0]
# optimal = minimize(
#     max_func, x, method="L-BFGS-B", bounds=[(None, None), (None, None), (-0.98, None)]
# )
# optimal

In [ ]:
optimal_data = dataset_to_optimize(optimal.x)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True, sharey=True)


im = ax.plot(times[:FOLD_SEGMENT], optimal_data)
ax.set_ylabel("Power (arb)")
ax.set_xlabel("Time [s]")


plt.suptitle("Power vs Time for Un-decayed Beamformed Data (Folded)", fontsize=16)

plt.show()

In [ ]:
# Dedisperse data


VELA_DM = 67.99


dedispersed_folded_dataset = dedisperse_dataset(
    output_weighted_folded, VELA_DM, 27 / 51_200_000, frequencies_axis_mhz, SEGMENT
)

In [ ]:
def max_func(x):
    return -dataset_to_optimize(x).max() * 1000


def dataset_to_optimize(x):
    theta_radians = THETA_RADIANS + x[0]
    phi_radians = PHI_RADIANS + x[1]

    antenna_phase_shift = calc_phase_shift_for_antennae(
        theta_radians, phi_radians, antenna_locations, REFERENCE_WAVELENGTH
    )
    antenna_phase_shift = antenna_phase_shift[np.newaxis, :]
    # Calculate weights
    rs = antenna_locations["r"].to_numpy()

    # trial this Gaussian weighting scheme - sigma chosen arbitrarily.
    SIGMA = 1 + x[2]
    weights = np.exp(-(rs**2) / (2 * SIGMA**2))
    weights = weights[np.newaxis, :]

    weighted_and_phase_shifted = weights * antenna_phase_shift
    weighted_beamformed_data = weighted_and_phase_shifted @ data
    output_weighted = get_fourier_power_spectrum(weighted_beamformed_data, SEGMENT)
    output_weighted_folded = get_folded_data(
        output_weighted, VELA_PERIOD, 27 / 51_200_000, SEGMENT
    )
    output_weighted_folded_dedispersed = dedisperse_dataset(
        output_weighted_folded, VELA_DM, 27 / 51_200_000, frequencies_axis_mhz, SEGMENT
    )

    output_weighted_folded_power = get_demeaned_scrunched_data(
        output_weighted_folded_dedispersed
    )

    std_noise = output_weighted_folded_power[
        output_weighted_folded_power.shape[0] // 2 :
    ].std(axis=0)
    output_weighted_folded_power /= std_noise

    return output_weighted_folded_power


x = [0, 0, 0]
# L-BFGS-B respects bounds while Nelder-Mead does not
optimal = minimize(
    max_func, x, method="L-BFGS-B", bounds=[(None, None), (None, None), (-0.98, None)]
)
optimal

In [ ]:
optimal_data = dataset_to_optimize(optimal.x)

fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True, sharey=True)


im = ax.plot(times[: FOLD_SEGMENT - 10], optimal_data)
ax.set_ylabel("Power (arb)")
ax.set_xlabel("Time [s]")


plt.suptitle(
    "Power vs Time for Un-decayed Beamformed Data (Folded & De-dispersed)", fontsize=16
)

plt.show()